In [2]:
import pandas as pd
np = pd.np
pd.set_option('display.max_rows', 4)
pd.set_option('display.max_columns', 500)

from IPython.display import display, HTML
import seaborn as sns
from seaborn.utils import plt

# display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML(open("../shared-resources/jupyter.css", "r").read()))
%matplotlib inline
# import mpld3

sns.mpl.rc("figure", figsize=(16, 6))
# plt.style.use('seaborn')
# plt.rc("figure", figsize=(16, 6))
# mpld3.enable_notebook()


from nlpia.data.loaders import get_data

df = pd.read_csv('../shared-resources/chatbot/testset.csv', index_col=None, header=0)
df.head()


,Context,Ground Truth Utterance,Distractor_0,Distractor_1,Distractor_2,Distractor_3,Distractor_4,Distractor_5,Distractor_6,Distractor_7,Distractor_8
0,anyone knows why my stock oneiric exports env ...,nice thanks! __eou__,"wrong channel for it, but check efnet.org, uno...","every time the kernel changes, you will lose v...",ok __eou__,!nomodeset > acer __eou__ I'm assuming it is a...,http://www.ubuntu.com/project/about-ubuntu/der...,thx __eou__ unfortunately the program isn't in...,how can I check? By doing a recovery for testi...,my humble apologies __eou__,#ubuntu-offtopic __eou__
1,i set up my hd such that i have to type a pass...,"so you dont know, ok, anyone else? __eou__ you...","nmap is nice, but it wasn't what I was looking...",ok __eou__,cdrom worked fine on windows. __eou__ i dont ...,"ah yes, i have read return as rerun __eou__",hm? __eou__,"not the case, LTS is every other .04 release. ...",Pretty much __eou__,I used the one I downloaded from AMD __eou__,"ffmpeg is part of the package , quixotedon , a..."
...,...,...,...,...,...,...,...,...,...,...,...
3,no suggestions? __eou__ links? __eou__ how can...,you cant load anything via usb or cd when luks...,-p sorry... __eou__ nmap -p22 __eou__ It d...,i guess so i can't even launch it. __eou__,noted __eou__,rxvt-unicode is one __eou__,I tarred all of ~ __eou__,I tarred all of ~ __eou__,"I don't really know if I can help, but I was c...","that works just fine, thanks! __eou__",thank you __eou__
4,I just added a second usb printer but not sure...,i was setting it up under the printer configur...,i'd say the most commonly venue would be via L...,"the old hardy man page, http://manpages.ubuntu...",i'll give a try __eou__,"by the way, the url you posted for davfs is fr...",http://ubuntuforums.org/showthread.php?t=15498...,"So I load up putty gui, then what do I do? __e...","you should read error messages, it says 'are ...",waiting the college semester to close just to ...,I was calling myself a jerk. All I know is tha...


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
tfidf

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [4]:
vectors = tfidf.fit_transform(['elephant hello world', 'elephant another time'])
vectors

<2x5 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [5]:
vectors = vectors.todense()
vectors

matrix([[ 0.        ,  0.44943642,  0.6316672 ,  0.        ,  0.6316672 ],
        [ 0.6316672 ,  0.44943642,  0.        ,  0.6316672 ,  0.        ]])

In [6]:
tfidf.vocabulary_

{'another': 0, 'elephant': 1, 'hello': 2, 'time': 3, 'world': 4}

In [7]:
tfidf = TfidfVectorizer(min_df=8, max_df=.3, max_features=100000)
tfidf

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.3, max_features=100000, min_df=8,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [8]:
tfidf.fit(pd.concat([df[df.columns[i]] for i in range(11)]))

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.3, max_features=100000, min_df=8,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [9]:
print(list(tfidf.vocabulary_)[:10])
print(len(tfidf.vocabulary_))

['anyone', 'knows', 'why', 'my', 'stock', 'oneiric', 'exports', 'env', 'var', 'username']
12358


In [10]:
X = tfidf.transform(df.Context)
X = X.todense()
X.shape

(18920, 12358)

In [11]:
y = tfidf.transform(df['Ground Truth Utterance']).todense()
y.shape

(18920, 12358)

In [12]:
def norm(x):
    """ 2-norm of a vector 
    >>> x = np.random.randn(10)
    >>> norm(x) == np.linalg.norm(x)
    """
    return np.sqrt(np.sum(x ** 2))

x = np.random.rand(10)
abs(norm(x) - np.linalg.norm(x)) / np.linalg.norm(x)

0.0

In [13]:
dots = np.dot(X.T, X).T
print(dots.shape)
print(X2.shape)
X2[:3], X[:3]

(12358, 12358)


NameError: name 'X2' is not defined

In [12]:
def get_statement(s='Hi'):
    q = np.array(tfidf.transform([s]).todense()[0])
    q = q / norm(q)
    dots = np.dot(q, X.T).T
    best_i = 0
    best_similarity = -1
    for i, similarity in enumerate(dots):
        # print(i, q, v)
        if similarity > best_similarity:
            best_similarity = similarity
            best_i = i
    return df.Context.iloc[best_i], best_i, best_similarity

In [13]:
get_statement('Hello Ubuntu')

('hello __eou__ __eot__ hello, can i help you? __eou__ __eot__ ',
 1022,
 matrix([[ 0.71722115]]))

In [14]:
df['Ground Truth Utterance'].iloc[get_statement("Ubuntu doesn't work on my Macbook Pro!")[1]]

'exactly my idea! __eou__'

In [15]:
def get_reply(s='Hi'):
    return df['Ground Truth Utterance'].iloc[get_statement(s)[1]]
    

In [16]:
get_reply('anyone knows why my stock oneiric exports env')

'nice thanks! __eou__'

In [17]:
get_reply('i set up my hd such that i have to type a pass')

'so you dont know, ok, anyone else? __eou__ you are like, yah my mouse doesnt work, reinstall your os lolol what a joke __eou__'

In [36]:
from sklearn.decomposition import PCA
pca = PCA(n_components=500)
pca = pca.fit(tfidf.transform(df.Context).todense())
X_100d = pca.transform(X)
y_100d = pca.transform(y)

In [37]:
def get_statement_100d(s='Hi'):
    q = pca.transform(tfidf.transform([s]).todense())[0] / norm(q)
    q = q / norm(q)
    dots = np.dot(q, X.T).T
    best_i = 0
    best_similarity = -1
    for i, similarity in enumerate(dots):
        # print(i, q, v)
        if similarity > best_similarity:
            best_similarity = similarity
            best_i = i
    
    return df.Context.iloc[best_i], best_i

def get_reply_100d(s='Hi'):
    return df['Ground Truth Utterance'].iloc[get_statement_100d(s)[1]]

In [24]:
print(get_statement_100d(df.Context[0])[0])
print(get_reply_100d(df.Context[0])[0])
print(get_reply_100d("I'm trying to use ubuntu on my macbook pro"))

anyone knows why my stock oneiric exports env var 'USERNAME'?  I mean what is that used for?  I know of $USER but not $USERNAME .  My precise install doesn't export USERNAME __eou__ __eot__ looks like it used to be exported by lightdm, but the line had the comment "// FIXME: Is this required?" so I guess it isn't surprising it is gone __eou__ __eot__ thanks!  How the heck did you figure that out? __eou__ __eot__ https://bugs.launchpad.net/lightdm/+bug/864109/comments/3 __eou__ __eot__ 
n
ok, i just figured i'd ask here incase I was just retarded lol __eou__


In [95]:
get_statement_100d("me just installed another serial port copier but don't know")

("I use cinnamon __eou__ But i don't know :P __eou__ __eot__ fair enough. I know Kazam has big issues in Gnome3 and cinnamon. __eou__ __eot__ ",
 10644)

In [96]:
get_statement_100d("I just added a second usb printer but not sure")

('obi Its not working without USB stick. Without USB stick it asks to select a boot medium. With USB stick, it boots correctly. __eou__ obi Still the fdisk shows its not bootable. __eou__ __eot__ That display is irrelevant since 15 years :) __eou__ __eot__ ',
 7989)

In [31]:

print(get_reply_100d("Did you like the movie Avatar?"))


and it still doesn't work? __eou__
